In [ ]:
# analysis/plot_spreads.ipynb

import pandas as pd
import matplotlib.pyplot as plt

# Load TOB snapshots
df = pd.read_csv("/opt/aud_arb/out/tob_snapshots.csv")

# Ensure timestamp is parsed
df["timestamp_iso"] = pd.to_datetime(df["timestamp_iso"])

# Separate by exchange
ir = df[df["exchange"] == "independentreserve"].copy()
kraken = df[df["exchange"] == "kraken"].copy()

# Align snapshots by timestamp (resample 10s)
ir = ir.set_index("timestamp_iso").resample("10s").first()
kraken = kraken.set_index("timestamp_iso").resample("10s").first()

# Merge to compare spreads
merged = pd.concat({"ir": ir, "kraken": kraken}, axis=1)

# Compute synthetic cross-exchange spread
merged["spread_cross"] = merged[("ir", "best_bid_price")] - merged[("kraken", "best_ask_price")]

# Plot
plt.figure(figsize=(12,6))
plt.plot(merged.index, merged["spread_cross"], label="IR bid - Kraken ask")
plt.axhline(0, color="black", linestyle="--")
plt.axhline(0.007 * merged[("kraken", "best_ask_price")].mean(), color="red", linestyle="--", label="0.7% threshold")
plt.title("Cross-Exchange Spread (Independent Reserve vs Kraken)")
plt.ylabel("Spread (AUD)")
plt.legend()
plt.show()
